In [4]:
import pandas as pd
import numpy as np
import nltk

In [2]:
labeled_tot = pd.read_excel('model_labeled.xlsx')

In [9]:
labeled_tot['tokens'] = labeled_tot.cleaned.apply(nltk.word_tokenize)

In [10]:
labeled_tot.head()

,Unnamed: 0,Unnamed: 0.1,Rating,Review,lang,cleaned,model_label,tokens
0,5,5,3,"I really like this game, I am currently on lev...",en,"I really like this game , I be currently on le...",5,"[I, really, like, this, game, ,, I, be, curren..."
1,6,6,3,I like it I can just never get good weapons th...,en,I like it I can just never get good weapon the...,5,"[I, like, it, I, can, just, never, get, good, ..."
2,10,10,5,I really love this game it contains strategy a...,en,I really love this game it contain strategy an...,0,"[I, really, love, this, game, it, contain, str..."
3,17,17,5,It good to play the game,en,It good to play the game,0,"[It, good, to, play, the, game]"
4,25,25,4,Nice game,en,Nice game,0,"[Nice, game]"


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
#from keras.layers.cudnn_recurrent import CuDNNLSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed
from keras.models import load_model
from keras.utils import Sequence
from keras.preprocessing.text import Tokenizer
import keras.backend as K
import gensim
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


In [14]:
t = Tokenizer()
t.fit_on_texts(labeled_tot.cleaned.tolist())
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(labeled_tot.cleaned.tolist())
max_length = labeled_tot.tokens.apply(len).max()
feats = encoded_docs
labels = to_categorical(labeled_tot.model_label)

X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
X_train = sequence.pad_sequences(X_train, maxlen=max_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_length)

In [ ]:
class_weight = compute_class_weight('balanced'
                                               ,[0,1,2,3,4,5]
                                               ,labeled_tot.model_label.apply(int).tolist())
model = Sequential()
model.add(Embedding(vocab_size, 256, 
                    input_length=max_length,
                    #input_shape = (None,max_length),
                    embeddings_regularizer = regularizers.l1(1e-5)))
model.add(Dropout(0.5))
#model.add(Conv1D(filters=700, kernel_size=2, padding='same', activation='tanh'))
#model.add(Conv1D(filters=7, kernel_size=2, padding='same', activation='tanh'))
#model.add(Conv1D(filters=50, kernel_size=3, padding='same', activation='sigmoid'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dropout(0.7))
model.add(LSTM(100, activation = 'tanh', kernel_regularizer = regularizers.l1(1e-5)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_hinge', optimizer='adam', metrics=['categorical_accuracy'])

checkpointer = ModelCheckpoint(filepath='weights.hdf5',
                               verbose=1, save_best_only=True,
                               monitor = 'val_categorical_accuracy')
model.fit(X_train, y_train, epochs=7, batch_size=100,
          validation_data = [X_test,y_test], callbacks=[checkpointer],
         class_weight = class_weight)

Train on 15825 samples, validate on 3957 samples
Epoch 1/7
15800/15825 [============================>.] - ETA: 0s - loss: 0.7183 - categorical_accuracy: 0.7201
Epoch 00001: val_categorical_accuracy improved from -inf to 0.73086, saving model to weights.hdf5
15825/15825 [==============================] - 464s 29ms/step - loss: 0.7178 - categorical_accuracy: 0.7202 - val_loss: 0.5558 - val_categorical_accuracy: 0.7309
Epoch 2/7
15800/15825 [============================>.] - ETA: 0s - loss: 0.5606 - categorical_accuracy: 0.7259
Epoch 00002: val_categorical_accuracy did not improve
15825/15825 [==============================] - 462s 29ms/step - loss: 0.5608 - categorical_accuracy: 0.7259 - val_loss: 0.5476 - val_categorical_accuracy: 0.7309
Epoch 3/7
13000/15825 [=======================>......] - ETA: 1:14 - loss: 0.5531 - categorical_accuracy: 0.7275